In [1]:
import numpy as np
import pandas as pd

In [2]:
cardio_data = pd.read_csv('data/cardio_data.csv', sep = ';')
cardio_data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
# changing column names
cardio_data.rename(columns = {'ap_hi': 'blood_pressure_max', 'ap_lo': 'blood_pressure_min', 'cholesterol': 'cholesterol_level', 'gluc': 'glucose_level', 'smoke': 'smoking_status', 'alco': 'alcohol_status', 'active': 'activity_level', 'cardio': 'disease'}, inplace = True)
# casting datatypes
cardio_data['height'] = cardio_data['height'].astype('float')
# converting age from days to years
cardio_data['age'] = cardio_data['age'] // 365
# calculating bmi
cardio_data['bmi'] = cardio_data['weight'] / (cardio_data['height'] / 100)**2

In [4]:
# changing sign of negative blood pressure values
cardio_data['blood_pressure_min'] = cardio_data['blood_pressure_min'].map(lambda x: -1 * x if x < 0 else x)
cardio_data['blood_pressure_max'] = cardio_data['blood_pressure_max'].map(lambda x: -1 * x if x < 0 else x)
# calculating blood pressure difference
cardio_data['blood_pressure_diff'] = cardio_data['blood_pressure_max'] - cardio_data['blood_pressure_min']
# dropping rows with diastolic blood pressure > systolic blood pressure
cardio_data = cardio_data.loc[cardio_data['blood_pressure_diff'] > 0]
# dropping rows with unreasonable bmi values
cardio_data = cardio_data.loc[(cardio_data['bmi'] > 10) & (cardio_data['bmi'] < 60)]
# dropping rows with unreasonable blood pressure values
cardio_data = cardio_data.loc[(cardio_data['blood_pressure_max'] > 70) & (cardio_data['blood_pressure_max'] < 200)]
cardio_data = cardio_data.loc[(cardio_data['blood_pressure_min'] > 40) & (cardio_data['blood_pressure_min'] < 160)]
# dropping duplicate values
cardio_data.drop_duplicates(inplace = True)
# dropping id column
cardio_data.drop(columns = ['id'], inplace = True)

In [5]:
cardio_data.to_csv('data/cardio_data_clean.csv')

In [6]:
cardio_data = pd.read_csv('data/cardio_data_clean.csv').drop(columns = ['Unnamed: 0'])
cardio_data.head()

,age,gender,height,weight,blood_pressure_max,blood_pressure_min,cholesterol_level,glucose_level,smoking_status,alcohol_status,activity_level,disease,bmi,blood_pressure_diff
0,50,2,168.0,62.0,110,80,1,1,0,0,1,0,21.967120,30
1,55,1,156.0,85.0,140,90,3,1,0,0,1,1,34.927679,50
2,51,1,165.0,64.0,130,70,3,1,0,0,0,1,23.507805,60
3,48,2,169.0,82.0,150,100,1,1,0,0,1,1,28.710479,50
4,47,1,156.0,56.0,100,60,1,1,0,0,0,0,23.011177,40


In [112]:
cardio_data['disease'].value_counts(normalize = True).round(3)
# very marginal class imbalance -> downsample minority class

0    0.506
1    0.494
Name: disease, dtype: float64

In [114]:
# downsampling minority class
no_disease = cardio_data.loc[cardio_data['disease'] == 0]
disease = cardio_data.loc[cardio_data['disease'] == 1]
no_disease_reduced = no_disease.sample(n = len(disease), random_state = 0)
cardio_data = pd.concat([disease, no_disease_reduced], axis = 0)

In [125]:
cardio_data.groupby('gender')['disease'].mean().round(3)

gender
1    0.498
2    0.505
Name: disease, dtype: float64

In [126]:
cardio_data.groupby('alcohol_status')['disease'].mean().round(3)

alcohol_status
0    0.501
1    0.481
Name: disease, dtype: float64

In [127]:
cardio_data.groupby('smoking_status')['disease'].mean().round(3)

smoking_status
0    0.503
1    0.474
Name: disease, dtype: float64

In [128]:
cardio_data.groupby('activity_level')['disease'].mean().round(3)

activity_level
0    0.538
1    0.491
Name: disease, dtype: float64

In [129]:
cardio_data.groupby('cholesterol_level')['disease'].mean().round(3)

cholesterol_level
1    0.441
2    0.601
3    0.766
Name: disease, dtype: float64

In [130]:
cardio_data.groupby('glucose_level')['disease'].mean().round(3)

glucose_level
1    0.481
2    0.592
3    0.622
Name: disease, dtype: float64

In [131]:
min_age, max_age = cardio_data['age'].min(), cardio_data['age'].max()
cardio_data['age_category'] = pd.cut(cardio_data['age'], bins = [min_age,40,50,60,max_age], labels = ['under 40', '40-50', '50-60', 'over 60'])
cardio_data.groupby('age_category')['disease'].mean().round(3)

age_category
under 40    0.229
40-50       0.393
50-60       0.534
over 60     0.700
Name: disease, dtype: float64

In [135]:
min_bmi, max_bmi = cardio_data['bmi'].min(), cardio_data['bmi'].max()
cardio_data['bmi_zone'] = pd.cut(cardio_data['bmi'], bins = [min_bmi,17.5,25,30,max_bmi], labels = ['underweight', 'normal weight', 'overweight', 'obese'])
cardio_data.groupby('bmi_zone')['disease'].mean().round(3)

bmi_zone
underweight      0.271
normal weight    0.402
overweight       0.511
obese            0.629
Name: disease, dtype: float64

In [133]:
min_diastolic_bp, max_diastolic_bp = cardio_data['blood_pressure_min'].min(), cardio_data['blood_pressure_min'].max()
cardio_data['diastolic_zone'] = pd.cut(cardio_data['blood_pressure_min'], bins = [min_diastolic_bp,80,90,120,max_diastolic_bp], labels = ['normal', 'stage 1', 'stage 2', 'stage 3'])
cardio_data.groupby('diastolic_zone')['disease'].mean().round(3)

diastolic_zone
normal     0.390
stage 1    0.745
stage 2    0.837
stage 3    0.875
Name: disease, dtype: float64

In [134]:
min_systolic_bp, max_systolic_bp = cardio_data['blood_pressure_max'].min(), cardio_data['blood_pressure_max'].max()
cardio_data['systolic_zone'] = pd.cut(cardio_data['blood_pressure_max'], bins = [min_systolic_bp,120,140,180,max_systolic_bp], labels = ['normal', 'stage 1', 'stage 2', 'stage 3'])
cardio_data.groupby('systolic_zone')['disease'].mean().round(3)

systolic_zone
normal     0.322
stage 1    0.702
stage 2    0.861
stage 3    0.843
Name: disease, dtype: float64

In [ ]:
{'female': 0.498, 'male': 0.505, 'non alcohol drinker': 0.501, 'alcohol drinker': 0.481, 'non active': 0.538, 'active': 0.491, 'normal cholesterol': 0.441, 'high cholesterol': 0.601, 'very high cholesterol': 0.766, 'normal glucose': 0.481, 'high glucose': 0.592, 'very high glucose': 0.622, 'under 40 years': 0.229, '40-50 years': 0.393, '50-60 years': 0.534, 'over 60 years': 0.7, 'underweight': 0.271, 'normal weight': 0.402, 'overweight': 0.511, 'obese': 0.629}